In [1]:
%load_ext autoreload
%autoreload 2

import logging
from urllib.parse import urljoin
from io import StringIO
from typing import Sequence
from datetime import date, datetime

import requests
import pandas as pd
import numpy as np

from etf_scraper.base import SecurityListing
from etf_scraper.scrapers import ISharesListings, SSGAListings

logger = logging.getLogger(__name__)


ssga_listings = SSGAListings.retrieve_listings()        
ishares_listings = ISharesListings.retrieve_listings()

#TODO: Vanguard, Invesco - get proof of concept for one of ishares, ssga holdings first

/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:254: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "inception_date"] = ssga_web_data_df_[
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:257: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "net_assets"] = ssga_web_data_df_["net_assets"].apply(
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:97: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set th

In [2]:
ssga_listings.tail()

,fund_name,ticker,product_url,inception_date,region,net_assets,exchange,fund_type,asset_class,cusip,isin,benchmark,provider
205,The Industrial Select Sector SPDR® Fund,XLI,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.366421e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y704,US81369Y7040,Industrial Select Sector Index,SSGA
206,The Materials Select Sector SPDR® Fund,XLB,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,5.265610e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y100,US81369Y1001,Materials Select Sector Index,SSGA
207,The Real Estate Select Sector SPDR® Fund,XLRE,https://www.ssga.com/us/en/intermediary/etfs/f...,2015-10-07,US,4.885160e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y860,US81369Y8600,Real Estate Select Sector Index,SSGA
208,The Technology Select Sector SPDR® Fund,XLK,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,3.720545e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y803,US81369Y8030,Technology Select Sector Index,SSGA
209,The Utilities Select Sector SPDR® Fund,XLU,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.627237e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y886,US81369Y8865,Utilities Select Sector Index,SSGA


In [6]:
ssga_listings['product_url'].iloc[-1]

'https://www.ssga.com/us/en/intermediary/etfs/funds/the-utilities-select-sector-spdr-fund-xlu'

In [7]:
ssga_listings

,fund_name,ticker,product_url,inception_date,region,net_assets,exchange,fund_type,asset_class,cusip,isin,benchmark,provider
0,State Street Aggregate Bond Index Fund - Class A,SSFCX,https://www.ssga.com/us/en/intermediary/ic/fun...,2014-09-19,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
1,State Street Aggregate Bond Index Fund - Class I,SSFDX,https://www.ssga.com/us/en/intermediary/ic/fun...,2014-09-19,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
2,State Street Aggregate Bond Index Fund - Class K,SSFEX,https://www.ssga.com/us/en/intermediary/ic/fun...,2014-09-19,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
3,State Street Emerging Markets Equity Index Fun...,SSKEX,https://www.ssga.com/us/en/intermediary/ic/fun...,2015-12-18,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
4,State Street Equity 500 Index Fund - Administr...,STFAX,https://www.ssga.com/us/en/intermediary/ic/fun...,2001-04-18,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,The Industrial Select Sector SPDR® Fund,XLI,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.366421e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y704,US81369Y7040,Industrial Select Sector Index,SSGA
206,The Materials Select Sector SPDR® Fund,XLB,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,5.265610e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y100,US81369Y1001,Materials Select Sector Index,SSGA
207,The Real Estate Select Sector SPDR® Fund,XLRE,https://www.ssga.com/us/en/intermediary/etfs/f...,2015-10-07,US,4.885160e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y860,US81369Y8600,Real Estate Select Sector Index,SSGA
208,The Technology Select Sector SPDR® Fund,XLK,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,3.720545e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y803,US81369Y8030,Technology Select Sector Index,SSGA


In [35]:
ticker = 'SSFOX'.lower()
url = (
    "https://www.ssga.com/us/en/intermediary/etfs/library-content/products/fund-data/"
    f"etfs/us/holdings-daily-us-en-{ticker}.xlsx"
)
url = (
    "https://www.ssga.com/us/en/intermediary/ic/library-content/products/fund-data/"
    f"mf/us/holdings-monthly-us-en-{ticker}.xlsx"
)
r = requests.get(url)
r.raise_for_status()

df = pd.read_excel(r.content)

In [36]:
df

,Fund Name:,State Street Target Retirement Fund,Unnamed: 2
0,Holdings:,As of 30-Nov-2022,NaN
1,Name,Ticker,Weight
2,State Street Aggregate Bond Index Portfolio,SSAFX,19.240308
3,SPDR Bloomberg 1-10 Year TIPS ETF,TIPX,17.316107
4,State Street Equity 500 Index II Portfolio,SSEYX,16.686813
5,SPDR Portfolio Short Term Treasury ETF,SPTS,14.936716
6,State Street Global All Cap Equity ex-US Index...,SSGVX,12.548396
7,SPDR Bloomberg High Yield Bond ETF,JNK,6.994789
8,SPDR Dow Jones Global Real Estate ETF,RWO,5.206697
9,SPDR Portfolio Short Term Corporate Bond ETF,SPSB,3.797775


In [33]:
ssga_listings[ssga_listings['fund_type']=='MF'].tail()

,fund_name,ticker,product_url,inception_date,region,net_assets,exchange,fund_type,asset_class,cusip,isin,benchmark,provider
64,State Street Target Retirement Fund - Class K,SSFOX,https://www.ssga.com/us/en/intermediary/ic/fun...,2014-09-30,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
65,State Street Target Retirement Fund - Class R3,SSFQX,https://www.ssga.com/us/en/intermediary/ic/fun...,2014-09-30,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
66,State Street Total Return V.I.S. Fund - Class 1,SSTIX,https://www.ssga.com/us/en/intermediary/ic/fun...,1985-07-01,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
67,State Street Total Return V.I.S. Fund - Class 3,SSTTX,https://www.ssga.com/us/en/intermediary/ic/fun...,2006-05-01,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
68,State Street U.S. Equity V.I.S. Fund - Class 1,SSUSX,https://www.ssga.com/us/en/intermediary/ic/fun...,1995-01-03,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA


In [9]:
b = ishares_listings.iloc[107]
a = SecurityListing()
a.ticker = b.ticker
a.product_url = b.product_url

In [10]:
df = ISharesListings.retrieve_holdings(a, datetime.strptime('20131220', "%Y%m%d").date())
df

,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,AAPL*,APPLE INC,Information Technology,Equity,1.594925e+09,3.07,NaN,2906735.0,548.70,United States,Bolsa Mexicana De Valores,USD,1.0,USD,NaN,IVV,2013-12-20
1,XOM,EXXON MOBIL CORP,Energy,Equity,1.389716e+09,2.68,NaN,14083058.0,98.68,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2013-12-20
2,GOOGL,GOOGLE INC CLASS A,Information Technology,Equity,9.851003e+08,1.90,NaN,895025.0,1100.64,United States,Spot Regulated Market - Bvb,USD,1.0,USD,NaN,IVV,2013-12-20
3,MSFT,MICROSOFT CORP,Information Technology,Equity,8.917892e+08,1.72,NaN,24253174.0,36.77,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2013-12-20
4,GEC,GENERAL ELECTRIC,Industrials,Equity,8.914768e+08,1.72,NaN,32583217.0,27.36,United States,Deutsche Boerse Xetra,USD,1.0,USD,NaN,IVV,2013-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,TER,TERADYNE INC,Information Technology,Equity,1.025474e+07,0.02,NaN,612955.0,16.73,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2013-12-20
497,UNS1,VIAVI SOLUTIONS INC,Information Technology,Equity,9.536256e+06,0.02,NaN,760467.0,12.54,United States,Boerse Stuttgart,USD,1.0,USD,NaN,IVV,2013-12-20
498,JBL,JABIL INC,Information Technology,Equity,9.517345e+06,0.02,NaN,590406.0,16.12,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2013-12-20
499,GHC,GRAHAM HOLDINGS COMPANY CLASS B,Consumer Discretionary,Equity,9.039303e+06,0.02,NaN,14046.0,643.55,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2013-12-20


In [9]:
# # query_dates = pd.bdate_range('2022-01-01', '2023-01-01')

# dfs = {}

# for query_date in query_dates[:10]:
#     r = get_holdings(query_date)
#     dfs[query_date] = pd.read_csv(StringIO(r.content.decode()), skiprows=9, skipfooter=2)

/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:254: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "inception_date"] = ssga_web_data_df_[
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:257: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "net_assets"] = ssga_web_data_df_["net_assets"].apply(


In [2]:
ssga_listings

,fund_name,ticker,product_url,inception_date,region,net_assets,exchange,fund_type,asset_class,cusip,isin,benchmark,provider
0,State Street Aggregate Bond Index Fund - Class A,SSFCX,/us/en/intermediary/ic/funds/state-street-aggr...,2014-09-19,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
1,State Street Aggregate Bond Index Fund - Class I,SSFDX,/us/en/intermediary/ic/funds/state-street-aggr...,2014-09-19,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
2,State Street Aggregate Bond Index Fund - Class K,SSFEX,/us/en/intermediary/ic/funds/state-street-aggr...,2014-09-19,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
3,State Street Emerging Markets Equity Index Fun...,SSKEX,/us/en/intermediary/ic/funds/state-street-emer...,2015-12-18,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
4,State Street Equity 500 Index Fund - Administr...,STFAX,/us/en/intermediary/ic/funds/state-street-equi...,2001-04-18,NaN,NaN,NaN,MF,NaN,NaN,NaN,NaN,SSGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,The Industrial Select Sector SPDR® Fund,XLI,/us/en/intermediary/etfs/funds/the-industrial-...,1998-12-16,US,1.366421e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y704,US81369Y7040,Industrial Select Sector Index,SSGA
206,The Materials Select Sector SPDR® Fund,XLB,/us/en/intermediary/etfs/funds/the-materials-s...,1998-12-16,US,5.265610e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y100,US81369Y1001,Materials Select Sector Index,SSGA
207,The Real Estate Select Sector SPDR® Fund,XLRE,/us/en/intermediary/etfs/funds/the-real-estate...,2015-10-07,US,4.885160e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y860,US81369Y8600,Real Estate Select Sector Index,SSGA
208,The Technology Select Sector SPDR® Fund,XLK,/us/en/intermediary/etfs/funds/the-technology-...,1998-12-16,US,3.720545e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y803,US81369Y8030,Technology Select Sector Index,SSGA


In [264]:
r = requests.get('https://investor.vanguard.com/investment-products/list/funddetail', headers={"user-agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
r_ = r.json()

In [278]:
r_['fund']['entity'][0]#.keys()

dict_keys(['type', 'profile', 'minimum', 'risk', 'fees', 'link', 'dailyPrice', 'yield', 'ytd', 'monthEndAvgAnnualRtn'])

In [283]:
r_['fund']['entity'][0]#['profile']

{'type': 'priceMonthEndPerformance',
 'profile': {'fundId': '0924',
  'ticker': 'BSV',
  'instrumentId': 2845959,
  'shortName': 'Short-Term Bond ETF',
  'longName': 'Vanguard Short-Term Bond ETF',
  'cusip': '921937827',
  'IOVTicker': 'BSV.IV',
  'inceptionDate': '2007-04-03T00:00:00-04:00',
  'newspaperAbbreviation': 'Short-Term Bond     ',
  'style': 'Bond Funds',
  'type': 'Short-Term Bond',
  'category': 'Short-Term Bond',
  'customizedStyle': 'Bond - Short-term Investment',
  'fixedIncomeInvestmentStyleId': '1',
  'fixedIncomeInvestmentStyleName': 'Short-term Treasury',
  'secDesignation': '',
  'maximumYearlyInvestment': '',
  'expenseRatio': '0.0400',
  'expenseRatioAsOfDate': '2022-04-29T00:00:00-04:00',
  'isInternalFund': True,
  'isExternalFund': False,
  'isMutualFund': False,
  'isETF': True,
  'isVLIP': False,
  'isVVAP': False,
  'is529': False,
  'hasAssociatedInvestorFund': True,
  'hasMoreThan1ShareClass': True,
  'isPESite': False,
  'fundFact': {'isActiveFund': Tr